<a href="https://colab.research.google.com/github/eli-shea/csci-1470-Final-Project/blob/master/DL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


Data Preprocessing

In [ ]:
def get_abbrev():
  team_dict1 = {}
  team_dict2 = {}
  url = 'https://raw.githubusercontent.com/fantasydatapros/data/master/misc/abbrev.csv'
  df = pd.read_csv(url)
  chargers_index = 9
  rams_index = 19
  for index, row in df.iterrows():
    label = index
    if index == 32:
      label = chargers_index
    elif index == 33:
      label = rams_index
    team_dict1[row['Team']] = label
    team_dict2[row['Abbrev']] = label
  return team_dict1, team_dict2
team_dict1, team_dict2 = get_abbrev()

In [ ]:
team_dict1

{'Arizona Cardinals': 28,
 'Atlanta Falcons': 20,
 'Baltimore Ravens': 1,
 'Buffalo Bills': 26,
 'Carolina Panthers': 0,
 'Chicago Bears': 23,
 'Cincinnati Bengals': 18,
 'Cleveland Browns': 8,
 'Dallas Cowboys': 4,
 'Denver Broncos': 27,
 'Detroit Lions': 15,
 'Green Bay Packers': 3,
 'Houston Texans': 13,
 'Indianapolis Colts': 25,
 'Jacksonville Jaguars': 22,
 'Kansas City Chiefs': 7,
 'Los Angeles Chargers': 9,
 'Los Angeles Rams': 19,
 'Miami Dolphins': 21,
 'Minnesota Vikings': 5,
 'New England Patriots': 24,
 'New Orleans Saints': 6,
 'New York Giants': 16,
 'New York Jets': 29,
 'Oakland Raiders': 14,
 'Philadelphia Eagles': 17,
 'Pittsburgh Steelers': 31,
 'San Diego Chargers': 9,
 'San Francisco 49ers': 12,
 'Seattle Seahawks': 10,
 'St. Louis Rams': 19,
 'Tampa Bay Buccaneers': 11,
 'Tennessee Titans': 2,
 'Washington Redskins': 30}

In [ ]:
def get_def(year):
  url = 'https://www.pro-football-reference.com'
  r = requests.get(url + '/years/' + str(year) + '/fantasy-points-against-WR.htm')
  soup = BeautifulSoup(r.content, 'html.parser')
  table = soup.find_all('table')[0]  
  df = pd.read_html(str(table))[0]
  
  df.columns = df.columns.droplevel(level=0)
  df = df.loc[df['Tm'] != 'Tm']
  df = df.loc[df['Tm'] != '']
  df = df.rename(columns={'Tm': 'Def', 
                          'Tgt': 'Tgt_Allowed', 
                          'Yds': 'ReceivingYds_Allowed', 
                          'Rec': 'Rec_Allowed', 
                          'Td': 'ReceivingTD_Allowed'})
  df = df.drop(columns=['FantPt', 'DKPt', 'FDPt','G'])
  df = df.replace({'Def' : team_dict1})
  return df


In [ ]:
def get_opponent_dicts(year):
  opponents_dict = {}
  url = 'https://www.pro-football-reference.com'
  r = requests.get(url + '/years/' + str(year) + '/games.htm')
  soup = BeautifulSoup(r.content, 'html.parser')
  table = soup.find_all('table')[0]  
  df = pd.read_html(str(table))[0]
  df = df.loc[df['Week'] != 'Week']
  df = df.loc[df['Week'] != 'WildCard']
  df = df.loc[df['Week'] != 'Division']
  df = df.loc[df['Week'] != 'ConfChamp']
  df = df.loc[df['Week'] != 'SuperBowl']
  df = df.rename(columns={'Winner/tie': 'T1', 'Loser/tie': 'T2'})
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  df = df.drop(columns=['Day','Date','Time', 'PtsW', 'PtsL', 'YdsW', 'TOW' , 'YdsL' , 'TOL'])
  df = df.replace({'T1': team_dict1, 'T2': team_dict1})
  d = [v for k,v in df.groupby('Week')]
  for frame in d:
    week_dict = {}
    week = 0
    for _, row in frame.iterrows():
      week_dict[row['T1']] = row['T2']
      week_dict[row['T2']] = row['T1']
      week = row['Week']
    opponents_dict[week] = week_dict

  return opponents_dict

In [ ]:
biglist = []
count = 0

for y in range(1999, 2019):
  opponents_dict = get_opponent_dicts(y)
  defenses = get_def(y-1)
  for w in range(1,17):
    week = 'week' + str(w)
    url = 'https://raw.githubusercontent.com/fantasydatapros/data/master/weekly/' + str(y) + '/' + week + '.csv'
    df = pd.read_csv(url)
    wr_filter = df.loc[:,('Pos')] == 'WR' 
    df = df[wr_filter]
    df = df.loc[:,('Player', 'Tm', 'RushingAtt', 'RushingYds', 'RushingTD', 'Rec', 'Tgt', 
            'ReceivingYds', 'ReceivingTD', 'PPRFantasyPoints')]
    df.loc[:,('Year')] = y
    df.loc[:,('Week')] = w
    df = df.replace({'Tm': team_dict2})
    df['Def'] = df['Tm'].map(opponents_dict[str(w)])
    df = df.merge(defenses, on='Def')
    if y == 2002:
      df = df[df.Tm != 'HOU']
      df = df[df.Def != 'HOU']
    biglist = biglist + df.values.tolist()
data= pd.DataFrame(biglist, columns=['Player', 'Tm', 'RushingAtt', 'RushingYds', 'RushingTD', 'Rec', 'Tgt', 
            'ReceivingYds', 'ReceivingTD', 'PPRFantasyPoints', 'Year', 'Week', 
            'Def', 'Tgt_Allowed', 'ReceivingYds_Allowed', 'Rec_Allowed', 'ReceivingTD_Allowed'])
defenses.head()
data.head()

,Player,Tm,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,PPRFantasyPoints,Year,Week,Def,Tgt_Allowed,ReceivingYds_Allowed,Rec_Allowed,ReceivingTD_Allowed
0,Jacquez Green,11,1.0,15.0,0.0,3.0,7.0,27.0,0.0,7.2,1999,1,16.0,227,105,1562,5
1,Bert Emanuel,11,0.0,0.0,0.0,3.0,12.0,53.0,0.0,8.3,1999,1,16.0,227,105,1562,5
2,Karl Williams,11,0.0,0.0,0.0,2.0,3.0,14.0,0.0,3.4,1999,1,16.0,227,105,1562,5
3,Reidel Anthony,11,0.0,0.0,0.0,2.0,4.0,10.0,0.0,3.0,1999,1,16.0,227,105,1562,5
4,Darnell McDonald,11,0.0,0.0,0.0,1.0,1.0,6.0,0.0,1.6,1999,1,16.0,227,105,1562,5


In [ ]:
data.sort_values(by=['Player', 'Year', 'Week'], inplace=True)
teams = sorted(data['Tm'].unique())
d = [v for k,v in data.groupby('Player', as_index=False)]

In [ ]:
data.columns

Index(['Player', 'Tm', 'RushingAtt', 'RushingYds', 'RushingTD', 'Rec', 'Tgt',
       'ReceivingYds', 'ReceivingTD', 'PPRFantasyPoints', 'Year', 'Week',
       'Def', 'Tgt_Allowed', 'ReceivingYds_Allowed', 'Rec_Allowed',
       'ReceivingTD_Allowed'],
      dtype='object')

In [ ]:
#Pass in a dataframe and list of columns to generate EWMAs on, as well as lookback window, an integer
#Returns dataframe with EWMAs generated
def generate_ewma(df, col_list, lookback_window):
  name_df = df.sort_values(by=['Player', 'Year', 'Week'])
  
  ewma_df = name_df.copy()
  for col in col_list:
    new_col_name = col + '_EWMA'
    ewma_df[new_col_name] = np.nan
  
  new_col_list = [col + '_EWMA' for col in col_list]
  
  player_list = [v for k,v in name_df.groupby('Player', as_index=False)]
  for player_df in player_list:
    print(player_df['Player'])
    year_list = [v for k,v in player_df.groupby('Year', as_index=False)]
    for year_df in year_list:
      year_df[new_col_list] = year_df[col_list].rolling(lookback_window).mean().shift(1)
      ewma_df.update(year_df)

  
  return ewma_df

In [ ]:
def get_prev_week_data(df, col_list, lookback_window):
  name_df = df.sort_values(by=['Player', 'Year', 'Week'])
  updated_df = name_df.copy()
  new_col_list = []
  
  for i in range(1, lookback_window + 1):
    new_cols_per_year = []
    
    for col in col_list:
      new_col_name = col + '_' + str(i) + '_Weeks_Ago'
      new_cols_per_year.append(new_col_name)
      updated_df[new_col_name] = np.nan
    new_col_list.append(new_cols_per_year)



  player_list = [v for k,v in name_df.groupby('Player', as_index=False)]
  for player_df in player_list:
    print(player_df['Player'])
    year_list = [v for k,v in player_df.groupby('Year', as_index=False)]
    for year_df in year_list:
      for i in range(1, lookback_window + 1):
        cur_year_cols = new_col_list[i-1]
        year_df[cur_year_cols] = year_df[col_list].shift(i)
      updated_df.update(year_df)
  
  return updated_df

In [ ]:
col_list = ['RushingAtt', 'RushingYds', 'RushingTD', 'Rec', 'Tgt',
       'ReceivingYds', 'ReceivingTD', 'PPRFantasyPoints',
       'Def', 'Tgt_Allowed', 'ReceivingYds_Allowed', 'Rec_Allowed',
       'ReceivingTD_Allowed']
updated_df = get_prev_week_data(data, col_list, 3)

In [ ]:
updated_df
# updated_df.columns


In [ ]:

non_features = ['Player', 'Tm', 'Def', 'Year' , 'Week']
label = ['PPRFantasyPoints']

In [ ]:
updated_df.to_csv('/drive/My Drive/Colab Notebooks/data.csv')

In [ ]:
#Splits training and testing data
#Takes in our large dataframe and a list of years that will belong to testing data
#Returns training df and testing df
def train_test_split_by_year(df, testing_year_list):
  training_data = df[~df['Year'].isin(testing_year_list)]
  testing_data = df[df['Year'].isin(testing_year_list)]
  return training_data, testing_data

In [ ]:
import requests

In [ ]:
league_id = 555555
season    = 2017

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

url = 'https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/' + \
      str(league_id) + '/?seasonId=' + str(season)

data = []
print('Week ', end='')
for week in range(1, 17):
    print(week, end=' ')

    r = requests.get(url,
                 params={'view': 'mMatchup', 
                         'view': 'mMatchupScore',
                         'scoringPeriodId': week})
    d = r.json()
    
    for tm in d['teams']:
        tmid = tm['id']
        for p in tm['roster']['entries']:
            name = p['playerPoolEntry']['player']['fullName']
            slot = p['lineupSlotId']
            pos  = slotcodes[slot]

            # injured status (need try/exc bc of D/ST)
            inj = 'NA'
            try:
                inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # projected/actual points
            proj, act = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != week:
                    continue
                if stat['statSourceId'] == 0:
                    act = stat['appliedTotal']
                elif stat['statSourceId'] == 1:
                    proj = stat['appliedTotal']
            if pos == 'WR':
              data.append([
                  week, tmid, name, slot, pos, inj, proj, act
              ])
print('\nComplete.')

data = pd.DataFrame(data, 
                    columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])

Week 1 [{'draftDetail': {'drafted': True, 'inProgress': False}, 'gameId': 1, 'id': 555555, 'schedule': [{'away': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 72.72}, 'teamId': 21, 'tiebreak': 0.0, 'totalPoints': 72.72}, 'home': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 65.94}, 'teamId': 30, 'tiebreak': 0.0, 'totalPoints': 65.94}, 'id': 0, 'matchupPeriodId': 1, 'playoffTierType': 'NONE', 'winner': 'AWAY'}, {'away': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 98.78}, 'teamId': 22, 'tiebreak': 0.0, 'totalPoints': 98.78}, 'home': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 125.64}, 'teamId': 3, 'tiebreak': 0.0, 'totalPoints': 125.64}, 'id': 1, 'matchupPeriodId': 1, 'playoffTierType': 'NONE', 'winner': 'HOME'}, {'away': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 94.34}, 'teamId': 9, 'tiebreak': 0.0, 'totalPoints': 94.34}, 'home': {'adjustment': 0.0, 'pointsByScoringPeriod': {'1': 64.84}, 'teamId': 26, 'tiebreak': 0.0, 'totalPoints': 64.84}, 'id': 2, 'mat

TypeError: ignored